# 1. 파이썬 데이터 모델

- 다른 객체지향 언어와는 다르게, 파이썬은 자료형의 길이를 구할 때, `collection.len()` 이 아닌, `len(collection)` 을 사용한다.
- 이러한 모습들이 pythonic 한 부분이고, 파이썬 데이터 모델이라고 하며, 이번 챕터에서는 데이터 모델들에 대해서 알아본다.
- 데이터 모델은 일종의 `프레임 워크`로써, 파이썬을 설명하는 것이라고 생각할 수 있으며, 
- 시퀀스 , 반복자 , 함수 , 클래스 , 콘텍스트 , 등 언어 자체의 구성단위에 대한 인터페이스를 공식적으로 정의한다.

- 프레임워크에서 호출되는 메서드를 생성하는 데에는 시간이 많이 걸린다. 이에 따라서 파이썬 인터프리터는 특별 메서드를 호출해, 기본적인 객체 연산을 수행한다. 특별 메소드(special method)는 양쪽에 언더바`_` 를 가지고 있고, double under 를 줄여, 던더 메소드라고 불린다.
- 예를 들어, obj[key] 와 같은 , indexing 과 더 나아가 slicing 은 `__getitem__()` 이라는 특별메소드가 지원한다.

In [38]:
import collections 
from random import choice 

Card = collections.namedtuple('Card',['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self,position):
        return self._cards[position]

- 우선적으로, collections.namedtuple()을 이용해서 개별 카드를 나타내는 클래스를 구현하는 것을 볼 수 있다.
- namedtuple 을 통해서, 메소드를 가지지 않으면서 일련의 속성을 지닌 클래스를 만들 수 있다.

In [39]:
beer_card = Card('7','diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [41]:
deck = FrenchDeck()
print(len(deck)) #__len__ 메소드가 제공한다.
print(deck[0]) #__getitem__ 메소드가 제공한다.

52
Card(rank='2', suit='spades')


In [10]:
from random import choice 
for _ in range(3):
    print(choice(deck))

Card(rank='8', suit='diamonds')
Card(rank='Q', suit='clubs')
Card(rank='6', suit='spades')


### 특별 메서드를 통해 파이썬 데이터 모델을 사용할 때의 두 가지 장점을 알 수 있다.
- 표준 연산을 수행하기 위해 클래스 자체에서 구현한 임의 메서드명을 암기할 필요가 없다.
    - 즉, 시퀀스의 길이를 알기 위해서, 해당 메소드의 이름을 외우기 보다는(ex.collenctions.len()), 특별메소드가 지원하는 기능을 사용하면 된다. (ex.len(collections))
- 파이썬 표준 라이브러리에서 제공하는 풍부한 기능을 그대로 쓸 수 있다.(별도의 구현의 필요가 없다)

In [44]:
import collections 
from random import choice 

Card = collections.namedtuple('Card',['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank,suit) for suit in self.suits for rank in self.ranks]
        
#     def __len__(self):
#         return len(self._cards)
    
#     def __getitem__(self,position):
#         return self._cards[position]

deck = FrenchDeck()
print(len(deck)) #__len__ 메소드가 제공한다.
print(deck[0]) #__getitem__ 메소드가 제공한다.

TypeError: object of type 'FrenchDeck' has no len()

In [47]:
from random import choice 
for _ in range(3):
    print(choice(deck))
#random 함수에는 len함수가 필요하다!

TypeError: object of type 'FrenchDeck' has no len()

- 위의 예시에는 `__len__` 과 `__getitem__` 던더 메소드를 없애고 같은 연산을 한 결과이다. 
- 이와 같이 두 가지 특별 메서드를 구현함으로써 FrenchDeck 은 표준 파이썬 시퀀스처럼 작동하므로 반복 및 슬라이싱 등의 핵심 언어 기능 및 기타 연산들까지 (random.choice , reversed , sorted) 들과 같은 표준 라이브러리를 사용할 수 있다.

### 특별 메소드는 어떻게 사용되나?

- 일단, 던더 메소드는 우리가 호출하기 위함이 아닌, 파이썬 인터프리터가 호출하기 위한 것이다.
    - `len(list_obj)` 를 실행하면, 파이썬 인터프리터는 `list_obj.__len__()` 를 호출한다.

- 그렇다고 해서, 항상 len()을 실행했을 때, 같은 던더 메소드로 가는 것은 아니다. 
    - 메소드를 호출하는 것은 가장 빠른 방법이 아니다.
    - 실제로, list , str , bytearray 와 같은 내장 자료형의 len() 함수는 던더 메소드가 아닌, PyVartObject C 구조체의 ob_size 필드 값을 반환한다.

- 던더 메소드는 암묵적으로 실행되는 경우도 존재한다.
    - choice 함수를 실행하기 위해서, len() 함수를 호출하며, 이 함수는 다시 `__len__()` 메소드를 호출한다.

- 실제로 던더 메소드를 호출해야 하는 경우는 그리 많지 않고, 내장 함수를 호출하는 것이 좋다.

- 시퀀스를 흉내내기 위해서 특별 메서드가 널리 사용된다.

_________________________

### 수치형 흉내 내기
- + 과 같은 연산자에 사용자 정의 객체가 응답할 수 있게 해주는 몇몇 던더 메소드가 있다.
- 자세한 것은 나중에 다시 다루도록 하고, 간단한 예제를 통해 던더 메서드를 사용하는 방법을 알아보자

In [70]:
from math import hypot

class Vector():
    
    def __init__(self,x=0,y=0):
        self.x = x
        self.y = y
        
    def __repr__(self):
        # __repr__ 던더 메서드는 객체를 문자열로 표현하기 위해 repr() 내장 메서드에 의해 호출된다. 
        #해당 메소드가 없으면 Vector 객체는 콘솔에 바이너리의 형태로 출력된다.
        return 'Vector(%r,%r)' % (self.x , self.y)
    
    def __abs__(self):
        return hypot(self.x, self.y)
    
    def __bool__(self):
        #__bool__을 따로 선언해 주지 않고, bool(x)를 소스 코드로 실행하면, __len__() 던더 메서드가 실행되고, 길이가 0이 아니면,
        # True 를 반환해준다. 우리는 2차원 벡터의 유클리디언 길이가 0 이상이면 True 를 반환하게끔 만드려 했으므로 bool 던더를 따로 만든다.
        return bool(abs(self))
    
    def __add__(self,other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x,y)
    
    def __mul__(self,scalar):
        return Vector(self.x * scalar , self.y * scalar)
    
    #add 와 mul 산술 연산자는 모두 피연산자를 변경하지 않고, Vector 객체를 새롭게 만들어서 반환한다.

#### `__repr__` 와 `__str__` 의 차이점 
- 아래의 datetime 예시를 보면 알 수 있다시피, str은 readable 이 관건으로 유저를 위한 것이다.
- repr은 unambiguous 가 관건으로 개발자를 위한 것이다.
- 하지만 repr 가 str 을 포괄하는 관점으로 repr 사용을 우선시한다.

In [79]:
class Sic(object): 
    def __repr__(object): return 'foo'

print( str(Sic()))
print( repr(Sic()))

class Sic(object):
    def __str__(object): return 'foo'

print( str(Sic()))
print( repr(Sic()))


foo
foo
foo


In [77]:
import datetime
today = datetime.datetime.now()
print(str(today))
print(repr(today))

2018-12-17 16:49:00.333145
datetime.datetime(2018, 12, 17, 16, 49, 0, 333145)
